In [18]:
from dotenv import load_dotenv
from intelligence_layer.core import (
    Task,
    TaskSpan,
    CompleteInput,
    LuminousControlModel,
    InMemoryTracer

)
from aleph_alpha_client import Prompt
import random
load_dotenv()

True

# How to log and debug a task
Apart from the normal python tools for logging (e.g. `print`) and debugging (e.g. debugger), the Intelligence Layer offers logging and debugging via the `Tracer`.  
Here are several steps you can use to debug tasks with the trace feature:

-----
Most logging of a task (input, output, time) is done simply by inheriting from `Task`. This logs to a trace.

 - If you don't care about logging and tracing, use the `NoOpTracer`
 - To create custom logging messages in a trace use `task_span.log()`.
 - To map a complex execution flow of a task into a single trace, pass the `task_span` of the `do_run` to other execution methods (e.g. `Task.run()` or `model.complete()`). 
   - If the execution method is not provided by the intelligence layer, the tracing of input and output has to manually happen. See the implementation of `Task.run()` for an example.
 - Use the [trace viewer](../../../README.md#running-the-trace-viewer) to view and inspect the trace
   - Use and display an `InMemoryTracer` in a notebook to automatically send the trace data to the trace viewer
   - To create persistant traces use the `FileTracer` instead, which can manually be uploaded to the trace viewer

# Example

In [17]:
class DummyTask(Task[str, str]):
    def __init__(self, model: LuminousControlModel = LuminousControlModel()) -> None:
        self._model = model

    def do_run(self, input: str, task_span: TaskSpan) -> str:
        should_output = random.random()
        # log a custom message and value
        task_span.log(
            "My very important log message that logs a random value", should_output
        )
        if should_output > 0.5:
            model_input = CompleteInput(prompt=Prompt.from_text(input), temperature=0.2)
            # Create a trace tree by passing task_span to .run or .complete methods.
            completion = self._model.complete(model_input, task_span)
            return completion.completions[0].completion
        else:
            return "Nope!"


tracer = InMemoryTracer()
DummyTask().run("", tracer)

# display an InMemoryTracer in a notebook and send the data to the trace viewer
display(tracer)

pass

Trace viewer not found under http://localhost:5173.
Consider running it for a better viewing experience.
If it is, set `TRACE_VIEWER_URL` in the environment.


Trace
└── DummyTask
    ├── ╭───────────────────────────────────────────────── Input ─────────────────────────────────────────────────╮
    │   │ ""                                                                                                      │
    │   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
    ├── ╭──────────────────────── My very important log message that logs a random value ─────────────────────────╮
    │   │ 0.9113332494061679                                                                                      │
    │   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
    ├── _Complete
    │   ├── ╭─────────────────────────────────────────────── Input ───────────────────────────────────────────────╮
    │   │   │ {                                                                                                   │
    │   │   │   "prompt": {                                                                                       │
    │   │   │     "items": [                                                                                      │
    │   │   │       {                                                                                             │
    │   │   │         "text": "",                                                                                 │
    │   │   │         "controls": []                                                                              │
    │   │   │       }                                                                                             │
    │   │   │     ]                                                                                               │
    │   │   │   },                                                                                                │
    │   │   │   "temperature": 0.2                                                                                │
    │   │   │ }                                                                                                   │
    │   │   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────╯
    │   ├── ╭─────────────────────────────────────────────── Model ───────────────────────────────────────────────╮
    │   │   │ "luminous-base-control"                                                                             │
    │   │   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────╯
    │   └── ╭────────────────────────────────────────────── Output ───────────────────────────────────────────────╮
    │       │ {                                                                                                   │
    │       │   "model_version": "20240215",                                                                      │
    │       │   "completions": [                                                                                  │
    │       │     {                                                                                               │
    │       │       "completion": " The point of this passage is to describe the process of creating a new        │
    │       │ product. The passage begins by stating that the company is creating a new product, and then goes on │
    │       │ to describe the steps involved in the process. The passage also mentions that the company is using  │
    │       │ a new technology to create the product.",                                                           │
    │       │       "finish_reason": "end_of_text"                                                                │
    │       │     }                                                                                               │
    │       │   ],                                                                                                │
    │       │   "n